# <font color='red'>SEDES - Gerador de Dados</font>

# <font color='blue'>1.0 import</font>

In [1]:
import csv
import os
import pandas       as pd
import random       as rd
import datetime     as dt
import urllib.parse as ps

import names
import time

import sqlalchemy as sa

from   dateutil.relativedelta import relativedelta

# <font color='blue'>2.0 parâmetros de ambiente</font>

In [2]:
# campo de negócio no BI
ORGAO_GESTOR = 'orgaoGestorDaPoliticaDeAssistenciaSocial'
ORGANIZACAO_PPA = 'comoEstaOrganizadoOPPADaPASNoMunicipio'
TUP_INSTR_GESTAO = (
    'planoDecenalDeAssistenciaSocial-possui',
    'planoMunicipalDeAssistenciaSocial-possui',
    'PlanoMunicipaldeEducacaoPermanentedoSUAS-possui',
    'diagnosticoSocioterritorial-possui',
    'RelatoriodeGestaoDoExercicioAnterior-possui',
    'LeideRegulamentacaodoSUAS-possui'
)
AREAS_ESSENCIAIS = (
    'quaisAreasEssenciaisDoSUASEstaoImplantadosOficialmente-gestao_do_trabalho',
    'quaisAreasEssenciaisDoSUASEstaoImplantadosOficialmente-protecao_social_basica',
    'quaisAreasEssenciaisDoSUASEstaoImplantadosOficialmente-protecao_social_especial',
    'quaisAreasEssenciaisDoSUASEstaoImplantadosOficialmente-gestao_do_suas'
)
VS = (
    'oMunicipioDesenvolveQuaisAcoesDeVigilanciaSocioassistencial-levanta_dados_no_territorio_equipamento_da_pas',
    'oMunicipioDesenvolveQuaisAcoesDeVigilanciaSocioassistencial-analisa_dados_levantados',
    'oMunicipioDesenvolveQuaisAcoesDeVigilanciaSocioassistencial-sistematiza_dados_e_produz_relatorios',
    'oMunicipioDesenvolveQuaisAcoesDeVigilanciaSocioassistencial-dissemina_dados_e_ou_relatorios',
    'oMunicipioDesenvolveQuaisAcoesDeVigilanciaSocioassistencial-realiza_diagnostico_socioterritorial',
    'oMunicipioDesenvolveQuaisAcoesDeVigilanciaSocioassistencial-realiza_monitoramento_sistematico_dos_servicos_socioassistenciais',
    'oMunicipioDesenvolveQuaisAcoesDeVigilanciaSocioassistencial-realiza_reunioes_com_as_equipes_responsaveis_pela_execucao_dos_servicos_socioassistenciais'
)
PREFIXO_MONITORA_SISTEMAS = 'realizaMonitoramentoDaAlimentacaoDosDadosNosSistemasDaRedeSUASEDaPlataformaEstadualPelasDemaisAreas-'
MONITORA_SISTEMAS = (
    'CAD_SUAS',
    'RMA',
    'SISC',
    'E-CPF',
    'simpeti',
    'planoDeAcaoSAA',
    'demonstrativoSinteticoDeExecucaoFisicoFinanceiro',
    'alimentacaoDaPlataformaEstadualDeMonitoramentoDaPAS'
)
# MONITORA_SISTEMAS = (
#     'quemAlimentaOsDadosNosSistemasDaRedeSUAS-CAD_SUAS',
#     'quemAlimentaOsDadosNosSistemasDaRedeSUAS-RMA',
#     'quemAlimentaOsDadosNosSistemasDaRedeSUAS-SISC',
#     'quemAlimentaOsDadosNosSistemasDaRedeSUAS-E-PCF',
#     'quemAlimentaOsDadosNosSistemasDaRedeSUAS-SIMPETI',
#     'quemAlimentaOsDadosNosSistemasDaRedeSUAS-Plano_de_Acao_SAA',
#     'quemAlimentaOsDadosNosSistemasDaRedeSUAS-Demonstrativo_Sintetico_de_Execucao_Fisico_Financeiro',
#     'quemAlimentaOsDadosNosSistemasDaRedeSUAS-Alimentacao_da_Plataforma_Estadual_de_Monitoramento_da_PAS'
# )
ALIMENTA_SUAS = (
    'quemAlimentaOsDadosNosSistemasDaRedeSUAS-gestao',
    'quemAlimentaOsDadosNosSistemasDaRedeSUAS-protecao_social_basica',
    'quemAlimentaOsDadosNosSistemasDaRedeSUAS-protecao_social_especial',
    'quemAlimentaOsDadosNosSistemasDaRedeSUAS-protecao_social_outros'
)
ACOMP_ASSESS_PAS = (
    'quantidadeDeMunicipiosAcompanhadosRemotamente',
    'quantidadeDeMunicipiosVisitadosInLoco',
    'quantidadeDeMunicipiosComPlanoDeProvidenciasAcompanhados',
    'quantidadeDeNotasTecnicasElaboradasEPublicizadas',
    'quantidadeDeBoletinsElaboradosEPublicizados'
)
ACS_ESTADO = (
    'quantidadeMensalDeReunioesExtraordinariasDoCEAS',
    'quantidadeMensalDeReunioesOrdinariasDoCEAS',
    'quantidadeMensalDeResolucoesEmitidasDoCEAS',
    'quantidadeMensalDeReunioesTecnicasRealizadasPeloCEAS'
)
EQUIPE_VS = (
    {'prefixo': 'qualEquipeCompoeOSetorDaVigilanciaSocioassistencial', 
     'sufixo': 'nome', 'tipo': 'STRING'},
    {'prefixo': 'qualEquipeCompoeOSetorDaVigilanciaSocioassistencial', 
     'sufixo': 'escolaridade', 'tipo': 'STRING'},
    {'prefixo': 'qualEquipeCompoeOSetorDaVigilanciaSocioassistencial', 
     'sufixo': 'formacao', 'tipo': 'STRING'},
    {'prefixo': 'qualEquipeCompoeOSetorDaVigilanciaSocioassistencial', 
     'sufixo': 'funcaoCargo', 'tipo': 'STRING'},
    {'prefixo': 'qualEquipeCompoeOSetorDaVigilanciaSocioassistencial', 
     'sufixo': 'horasMediasDedicadasAVigilancia', 'tipo': 'INTEGER'}
)

# data inicial de simulação
DATA_INICIAL = (2018, 3, 1)

# tabelas gerais
N_ORGAO_GESTOR = (6, 11) # quantidade de orgãos gestores possíveis para a tabela de orgãos gestores
N_ESCOLARIDADE = (5, 10) # quantidade de escolaridades possíveis para a tabela escolaridade
N_FORMACAO     = (5, 10) # quantidade de formações possíveis para a tabela formacao
N_CARGO        = (5, 10) # quantidade de cargos possíveis para a tabela cargo
N_EQUIPE_VS    = (2, 6)  # quantidade de pessoas possíveis para compor a equipe de VS
N_CAPACITACOES_OFERTADAS = (0, 100) # quantidade possível de capacitações
N_ACOMP_ASSESS_PAS = (0, 217) # quantidade possível de municípios com acompanhamentos e assessoramento
N_ACS_ESTADO = (0, 10) # quantidade possível de municípios com acompanhamentos e assessoramento
# formulários respondidos
form_resp = (80, 100) # % de formulários respondidos
orgao_gestor = (1, 3) # possibilidades de orgão gestor por município a cada data de referência
campos_por_form = (50, 150) # quantidade de campos por formulário sem contar os identificados para o BI

rd.seed(55) # mantem sempre os mesmos parâmetros de randomização

In [3]:
CAPACITACOES_OFERTADAS = (
    'capacitacoesOfertadasGestaoSUAS-conselheiros-quantidadeEstado',
    'capacitacoesOfertadasGestaoSUAS-conselheiros-quantidadeGovernoFederal',
    'capacitacoesOfertadasGestaoSUAS-conselheiros-quantidadeMunicipio',
    'capacitacoesOfertadasGestaoSUAS-gestor-quantidadeEstado',
    'capacitacoesOfertadasGestaoSUAS-gestor-quantidadeGovernoFederal',
    'capacitacoesOfertadasGestaoSUAS-gestor-quantidadeMunicipio',
#     'capacitacoesOfertadasGestaoSUAS-nome',
#     'capacitacoesOfertadasGestaoSUAS-quantidadeEstado',
#     'capacitacoesOfertadasGestaoSUAS-quantidadeGovernoFederal',
#     'capacitacoesOfertadasGestaoSUAS-quantidadeMunicipio',
    'capacitacoesOfertadasGestaoSUAS-trabalhadoresDeNivelMedio-quantidadeEstado',
    'capacitacoesOfertadasGestaoSUAS-trabalhadoresDeNivelMedio-quantidadeGovernoFederal',
    'capacitacoesOfertadasGestaoSUAS-trabalhadoresDeNivelMedio-quantidadeMunicipio',
    'capacitacoesOfertadasGestaoSUAS-trabalhadoresDeNivelSuperior-quantidadeEstado',
    'capacitacoesOfertadasGestaoSUAS-trabalhadoresDeNivelSuperior-quantidadeGovernoFederal',
    'capacitacoesOfertadasGestaoSUAS-trabalhadoresDeNivelSuperior-quantidadeMunicipio',
    'capacitacoesOfertadasProtecaoSocialBasica-conselheiros-quantidadeEstado',
    'capacitacoesOfertadasProtecaoSocialBasica-conselheiros-quantidadeGovernoFederal',
    'capacitacoesOfertadasProtecaoSocialBasica-conselheiros-quantidadeMunicipio',
    'capacitacoesOfertadasProtecaoSocialBasica-gestor-quantidadeEstado',
    'capacitacoesOfertadasProtecaoSocialBasica-gestor-quantidadeGovernoFederal',
    'capacitacoesOfertadasProtecaoSocialBasica-gestor-quantidadeMunicipio',
#     'capacitacoesOfertadasProtecaoSocialBasica-nome',
#     'capacitacoesOfertadasProtecaoSocialBasica-quantidadeEstado',
#     'capacitacoesOfertadasProtecaoSocialBasica-quantidadeGovernoFederal',
#     'capacitacoesOfertadasProtecaoSocialBasica-quantidadeMunicipio',
    'capacitacoesOfertadasProtecaoSocialBasica-trabalhadoresDeNivelMedio-quantidadeEstado',
    'capacitacoesOfertadasProtecaoSocialBasica-trabalhadoresDeNivelMedio-quantidadeGovernoFederal',
    'capacitacoesOfertadasProtecaoSocialBasica-trabalhadoresDeNivelMedio-quantidadeMunicipio',
    'capacitacoesOfertadasProtecaoSocialBasica-trabalhadoresDeNivelSuperior-quantidadeEstado',
    'capacitacoesOfertadasProtecaoSocialBasica-trabalhadoresDeNivelSuperior-quantidadeGovernoFederal',
    'capacitacoesOfertadasProtecaoSocialBasica-trabalhadoresDeNivelSuperior-quantidadeMunicipio',
    'capacitacoesOfertadasProtecaoSocialControleSocial-conselheiros-quantidadeEstado',
    'capacitacoesOfertadasProtecaoSocialControleSocial-conselheiros-quantidadeGovernoFederal',
    'capacitacoesOfertadasProtecaoSocialControleSocial-conselheiros-quantidadeMunicipio',
    'capacitacoesOfertadasProtecaoSocialControleSocial-gestor-quantidadeEstado',
    'capacitacoesOfertadasProtecaoSocialControleSocial-gestor-quantidadeGovernoFederal',
    'capacitacoesOfertadasProtecaoSocialControleSocial-gestor-quantidadeMunicipio',
#     'capacitacoesOfertadasProtecaoSocialControleSocial-nome',
#     'capacitacoesOfertadasProtecaoSocialControleSocial-quantidadeEstado',
#     'capacitacoesOfertadasProtecaoSocialControleSocial-quantidadeGovernoFederal',
#     'capacitacoesOfertadasProtecaoSocialControleSocial-quantidadeMunicipio',
    'capacitacoesOfertadasProtecaoSocialControleSocial-trabalhadoresDeNivelMedio-quantidadeEstado',
    'capacitacoesOfertadasProtecaoSocialControleSocial-trabalhadoresDeNivelMedio-quantidadeGovernoFederal',
    'capacitacoesOfertadasProtecaoSocialControleSocial-trabalhadoresDeNivelMedio-quantidadeMunicipio',
    'capacitacoesOfertadasProtecaoSocialControleSocial-trabalhadoresDeNivelSuperior-quantidadeEstado',
    'capacitacoesOfertadasProtecaoSocialControleSocial-trabalhadoresDeNivelSuperior-quantidadeGovernoFederal',
    'capacitacoesOfertadasProtecaoSocialControleSocial-trabalhadoresDeNivelSuperior-quantidadeMunicipio',
    'capacitacoesOfertadasProtecaoSocialEspecial-conselheiros-quantidadeEstado',
    'capacitacoesOfertadasProtecaoSocialEspecial-conselheiros-quantidadeGovernoFederal',
    'capacitacoesOfertadasProtecaoSocialEspecial-conselheiros-quantidadeMunicipio',
    'capacitacoesOfertadasProtecaoSocialEspecial-gestor-quantidadeEstado',
    'capacitacoesOfertadasProtecaoSocialEspecial-gestor-quantidadeGovernoFederal',
    'capacitacoesOfertadasProtecaoSocialEspecial-gestor-quantidadeMunicipio',
#     'capacitacoesOfertadasProtecaoSocialEspecial-nome',
#     'capacitacoesOfertadasProtecaoSocialEspecial-quantidadeEstado',
#     'capacitacoesOfertadasProtecaoSocialEspecial-quantidadeGovernoFederal',
#     'capacitacoesOfertadasProtecaoSocialEspecial-quantidadeMunicipio',
    'capacitacoesOfertadasProtecaoSocialEspecial-trabalhadoresDeNivelMedio-quantidadeEstado',
    'capacitacoesOfertadasProtecaoSocialEspecial-trabalhadoresDeNivelMedio-quantidadeGovernoFederal',
    'capacitacoesOfertadasProtecaoSocialEspecial-trabalhadoresDeNivelMedio-quantidadeMunicipio',
    'capacitacoesOfertadasProtecaoSocialEspecial-trabalhadoresDeNivelSuperior-quantidadeEstado',
    'capacitacoesOfertadasProtecaoSocialEspecial-trabalhadoresDeNivelSuperior-quantidadeGovernoFederal',
    'capacitacoesOfertadasProtecaoSocialEspecial-trabalhadoresDeNivelSuperior-quantidadeMunicipio'
)
len(CAPACITACOES_OFERTADAS)

48

# <font color='blue'>3.0 extração</font>

## 3.1 extração de dados externo

In [4]:
# abrindo os arquivos...
caminho_in = '../dados/apoio/'
caminho_out = '../dados/simulacao/'

arq_municipios    = caminho_in + "tab_municipios_new.csv"
arq_instr_gestao  = caminho_in + "tab_instr_gestao.csv"
arq_escolaridades = caminho_in + "tab_escolaridade.csv"
arq_formacoes     = caminho_in + "tab_formacoes.csv"
arq_cargos        = caminho_in + "tab_cargos.csv"

with open(arq_escolaridades, encoding="utf8") as f:
    lst_tot_escolaridades = list(csv.reader(f))

with open(arq_formacoes, encoding="utf8") as f:
    lst_tot_formacoes = list(csv.reader(f))
    
with open(arq_cargos, encoding="utf8") as f:
    lst_tot_cargos = list(csv.reader(f))

df_municipios =   pd.read_csv(arq_municipios, sep = ',', dtype = str)
df_instr_gestao = pd.read_csv(arq_instr_gestao, sep = ',', dtype = str)

print("municipios:              linhas = %s , colunas = %s " % df_municipios.shape)
print("instrumentos de gestão:  linhas = %s , colunas = %s " % df_instr_gestao.shape)
print("escolaridades:           linhas = %s " % len(lst_tot_escolaridades))
print("formacoes:               linhas = %s " % len(lst_tot_formacoes))
print("cargos:                  linhas = %s " % len(lst_tot_cargos))

municipios:              linhas = 217 , colunas = 4 
instrumentos de gestão:  linhas = 5 , colunas = 1 
escolaridades:           linhas = 13 
formacoes:               linhas = 226 
cargos:                  linhas = 134 


In [5]:
df_municipios

,cod_ibge,municipio,regiao_planejamento,porte
0,2100055,Açailândia,24 – REGIÃO DOS CARAJÁS,Grande Porte
1,2100105,Afonso Cunha,07 – REGIÃO DO ALTO MUNIM,Porte I
2,2100154,Água Doce Do Maranhão,13 – REGIÃO DO DELTA DO PARNAÍBA,Porte I
3,2100204,Alcântara,20 – REGIÃO DO PERICUMÃ,Porte II
4,2100303,Aldeias Altas,32 – REGIÃO DOS TIMBIRAS,Porte II
...,...,...,...,...
212,2112803,Viana,30 – REGIÃO DOS LAGOS,Porte II
213,2112852,Vila Nova Dos Martírios,24 – REGIÃO DOS CARAJÁS,Porte I
214,2112902,Vitória Do Mearim,26 – REGIÃO DOS EIXOS RODOFERROVIÁRIOS,Porte II
215,2113009,Vitorino Freire,17 – REGIÃO DO MEARIM,Porte II


# <font color='blue'>4.0 transformação</font>

## 4.1 Tabelas de Negócio

### 4.1.1 tabela de "Orgãos Gestores"

In [6]:
# cria uma tabela de orgãos gestores possíveis
# quantidade de orgão gestores na tabela

lst_OG = ['Orgão Gestor ' + str(x + 1).zfill(2) for x in range(rd.randint(N_ORGAO_GESTOR[0], N_ORGAO_GESTOR[1]))]
lst_OG

['Orgão Gestor 01',
 'Orgão Gestor 02',
 'Orgão Gestor 03',
 'Orgão Gestor 04',
 'Orgão Gestor 05',
 'Orgão Gestor 06']

### 4.1.2 tabela de "Organização de PPAs"

In [7]:
lst_org_PPA = (
    'Por nível de proteção social',
    'Por programas, serviços, projetos e benefícios'
)
lst_org_PPA

('Por nível de proteção social',
 'Por programas, serviços, projetos e benefícios')

### 4.1.3 tabela de "Escolaridade"

In [8]:
# sorteia a quantidade de nomes de escolaridades irão para a lista
n = rd.randint(N_ESCOLARIDADE[0], N_ESCOLARIDADE[1])

# sorteia quais os nomes de escolaridade farão parte dos dados
lst_escolaridade = rd.sample(lst_tot_escolaridades, k=n)
lst_escolaridade

[['Fundamental - Completo'],
 ['Pós-graduação (nível doutor) - Incompleto'],
 ['Médio - Completo'],
 ['Fundamental - Incompleto'],
 ['Pós-graduação (nível doutor) - Completo'],
 ['Pós-graduação (nível mestrado) - Completo']]

### 4.1.4 tabela de "Formação"

In [9]:
# sorteia a quantidade de nomes de formações irão para a lista
n = rd.randint(N_FORMACAO[0], N_FORMACAO[1])

# sorteia quais os nomes de escolaridade farão parte dos dados
lst_formacao = rd.sample(lst_tot_formacoes, k=n)
lst_formacao

[['Cooperativismo'],
 ['Materiais'],
 ['Psicologia'],
 ['Engenharia Industrial Madeireira'],
 ['Jornalismo']]

### 4.1.5 tabela de "Cargo"

In [10]:
# sorteia a quantidade de nomes de cargos irão para a lista
n = rd.randint(N_CARGO[0], N_CARGO[1])

# sorteia quais os nomes de escolaridade farão parte dos dados
lst_cargo = rd.sample(lst_tot_cargos, k=n)
lst_cargo

[['Tecnico em Radiologia'],
 ['Servicos Gerais'],
 ['Mecanico'],
 ['Especialistas'],
 ['Sargento'],
 ['Bibliotecário'],
 ['Gerente'],
 ['Controlador'],
 ['Atendente de caixa']]

### 4.1.6 tabela "Município"

In [11]:
df_municipios = df_municipios[['municipio']]
df_municipios

,municipio
0,Açailândia
1,Afonso Cunha
2,Água Doce Do Maranhão
3,Alcântara
4,Aldeias Altas
...,...
212,Viana
213,Vila Nova Dos Martírios
214,Vitória Do Mearim
215,Vitorino Freire


### 4.1.7 tabela "Serviço/Formulário"

In [12]:
# lista de serviços (cada serviço é equivalente e um formulário)
tup_servicos = (
    ('Questionário Semestral - Município',  'semestral', 'municipios'),
    ('Questionário Semestral - SEDES', 'semestral', 'sedes')
)
df_tb_servicos = pd.DataFrame(tup_servicos, columns=['servico', 'periodicidade', 'alvo'])
df_tb_servicos

,servico,periodicidade,alvo
0,Questionário Semestral - Município,semestral,municipios
1,Questionário Semestral - SEDES,semestral,sedes


### 4.1.8 serviços/formulário x campos fictícios

In [13]:
# função que gera valores para campos fictícios
def GeraValor(tp):
    vl = '0'
    if tp == 'STRING':
        vl = 'valor_' + str(rd.randint(1, 1000))
    return vl

In [14]:
# lista de campos por serviço/formulário
cont = 0; lst_campos = []
for i in tup_servicos:
    n_campos = rd.randint(10, 15)
    for j in range(n_campos):
        cont += 1
        lst = [i[0], 'col_' + str(cont), 'STRING']
        lst_campos.append(lst)
lst_campos

df_campos_fic = pd.DataFrame(lst_campos, columns=['servico', 'atributo', 'tipo'])
df_campos_fic

,servico,atributo,tipo
0,Questionário Semestral - Município,col_1,STRING
1,Questionário Semestral - Município,col_2,STRING
2,Questionário Semestral - Município,col_3,STRING
3,Questionário Semestral - Município,col_4,STRING
4,Questionário Semestral - Município,col_5,STRING
5,Questionário Semestral - Município,col_6,STRING
6,Questionário Semestral - Município,col_7,STRING
7,Questionário Semestral - Município,col_8,STRING
8,Questionário Semestral - Município,col_9,STRING
9,Questionário Semestral - Município,col_10,STRING


## (A) Módulo Semestral

### A.1 gera datas base - semestral

In [15]:
# cria uma coleção de datas de referências de todos os semestres desde o início até hoje
# confirma que a data inicial é o primeiro dia do semestre
dt_inicial = dt.date(DATA_INICIAL[0], 1 if DATA_INICIAL[1] <= 6 else 7, 1)

# monta uma lista de todos os semestres desde a data inicial até hoje
lst_dt_semestral = []; dt_atual = dt_inicial
while dt_atual < dt.date.today():
    dtf = dt_atual + relativedelta(months = 6) - relativedelta(days = 1)
    n_dias = (dtf - dt_atual).days
    lst_dt_semestral.append([dt_atual, dtf, n_dias])
    dt_atual = dt_atual + relativedelta(months = 6)

# substitui a última data pela data de hoje
lst_dt_semestral[len(lst_dt_semestral) - 1][1] = dt.date.today()
lst_dt_semestral[len(lst_dt_semestral) - 1][2] = (lst_dt_semestral[len(lst_dt_semestral) - 1][1] - lst_dt_semestral[len(lst_dt_semestral) - 1][0]).days
lst_dt_semestral

[[datetime.date(2018, 1, 1), datetime.date(2018, 6, 30), 180],
 [datetime.date(2018, 7, 1), datetime.date(2018, 12, 31), 183],
 [datetime.date(2019, 1, 1), datetime.date(2019, 6, 30), 180],
 [datetime.date(2019, 7, 1), datetime.date(2019, 12, 31), 183],
 [datetime.date(2020, 1, 1), datetime.date(2020, 6, 30), 181],
 [datetime.date(2020, 7, 1), datetime.date(2020, 12, 31), 183],
 [datetime.date(2021, 1, 1), datetime.date(2021, 6, 30), 180],
 [datetime.date(2021, 7, 1), datetime.date(2021, 10, 2), 93]]

### A.2 BASE formulários MUNICÍPIOS

#### A.2.1 Protocolos/Dt_Protocolos/Municípios

In [16]:
# cria a combinação de municípios e serviços para alvo "municípios"
dfa = df_tb_servicos[df_tb_servicos['alvo'] == 'municipios'][['servico', 'periodicidade']]
df = pd.merge(dfa, df_municipios, how='cross')

# cria datas de protocolos
lst = []; pct = 1
for idx in range(df.shape[0]):
    for i in lst_dt_semestral:
        n_protocolos = rd.randint(0, 3)
        if n_protocolos > 0:
            lst.append([idx, 'pct_m_' + str(pct), i[0],
                        df.loc[idx, 'servico'], df.loc[idx, 'municipio']])
            pct += 1
            for j in range(1, n_protocolos):
                dias = rd.randint(1, i[2])
                dt = i[0] + relativedelta(days = dias)
                lst.append([idx, 'pct_m_' + str(pct), dt, 
                            df.loc[idx, 'servico'], df.loc[idx, 'municipio']])
                pct += 1
df_pct_mu = pd.DataFrame(lst, columns=['idx', 'protocolo', 'dt_protocolo', 'servico', 'municipio'])
lst_pct_mu = list(df_pct_mu['protocolo'])
display(df_pct_mu)

,idx,protocolo,dt_protocolo,servico,municipio
0,0,pct_m_1,2018-07-01,Questionário Semestral - Município,Açailândia
1,0,pct_m_2,2018-08-28,Questionário Semestral - Município,Açailândia
2,0,pct_m_3,2018-07-15,Questionário Semestral - Município,Açailândia
3,0,pct_m_4,2019-01-01,Questionário Semestral - Município,Açailândia
4,0,pct_m_5,2019-03-02,Questionário Semestral - Município,Açailândia
...,...,...,...,...,...
2659,216,pct_m_2660,2020-01-01,Questionário Semestral - Município,Zé Doca
2660,216,pct_m_2661,2020-05-21,Questionário Semestral - Município,Zé Doca
2661,216,pct_m_2662,2020-05-04,Questionário Semestral - Município,Zé Doca
2662,216,pct_m_2663,2021-07-01,Questionário Semestral - Município,Zé Doca


#### A.2.2 Campos Fictícios para servirços de respondedor município - (A.2.1) x (4.1.8)

In [17]:
# gera dataset principal a partir de campos fictícios e do dataset de protocolos
df_final = pd.merge(df_pct_mu, df_campos_fic).drop(['idx', 'municipio'], axis=1)

# atribui valor aos campos fictícios
df_final['valor'] = df_final['tipo'].apply(GeraValor)
df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_m_1,2018-07-01,Questionário Semestral - Município,col_1,STRING,valor_23
1,pct_m_1,2018-07-01,Questionário Semestral - Município,col_2,STRING,valor_102
2,pct_m_1,2018-07-01,Questionário Semestral - Município,col_3,STRING,valor_509
3,pct_m_1,2018-07-01,Questionário Semestral - Município,col_4,STRING,valor_328
4,pct_m_1,2018-07-01,Questionário Semestral - Município,col_5,STRING,valor_816
...,...,...,...,...,...,...
37291,pct_m_2664,2021-07-28,Questionário Semestral - Município,col_10,STRING,valor_359
37292,pct_m_2664,2021-07-28,Questionário Semestral - Município,col_11,STRING,valor_477
37293,pct_m_2664,2021-07-28,Questionário Semestral - Município,col_12,STRING,valor_507
37294,pct_m_2664,2021-07-28,Questionário Semestral - Município,col_13,STRING,valor_252


#### A.2.4 Gera linha/campo com atributo "município" e concatena com campos fictícios - (A.2.3)

In [18]:
# gera um dataset temporário de atributos municípios
df_tmp = df_pct_mu.copy()
df_tmp['atributo'] = 'municipio'
df_tmp['tipo'] = 'STRING'
df_tmp = df_tmp.rename(columns={'municipio': 'valor'})
df_tmp = df_tmp.drop(['idx'], axis=1)

# gera dataset final para aplicar os registros de negócio
df_final = pd.concat([df_final, df_tmp])
df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_m_1,2018-07-01,Questionário Semestral - Município,col_1,STRING,valor_23
1,pct_m_1,2018-07-01,Questionário Semestral - Município,col_2,STRING,valor_102
2,pct_m_1,2018-07-01,Questionário Semestral - Município,col_3,STRING,valor_509
3,pct_m_1,2018-07-01,Questionário Semestral - Município,col_4,STRING,valor_328
4,pct_m_1,2018-07-01,Questionário Semestral - Município,col_5,STRING,valor_816
...,...,...,...,...,...,...
2659,pct_m_2660,2020-01-01,Questionário Semestral - Município,municipio,STRING,Zé Doca
2660,pct_m_2661,2020-05-21,Questionário Semestral - Município,municipio,STRING,Zé Doca
2661,pct_m_2662,2020-05-04,Questionário Semestral - Município,municipio,STRING,Zé Doca
2662,pct_m_2663,2021-07-01,Questionário Semestral - Município,municipio,STRING,Zé Doca


### A.3 BASE formulários SEDES-MA

#### A.3.1 Protocolos/Dt_Protocolos/Sedes

In [19]:
# cria a combinação de municípios e serviços para alvo "SEDES"
df = df_tb_servicos[df_tb_servicos['alvo'] == 'sedes'][['servico', 'periodicidade']].reset_index()
df['respondedor'] = 'sedes'

# cria datas de protocolos
lst = []; pct = 1
for idx in range(df.shape[0]):
    for i in lst_dt_semestral:
        n_protocolos = rd.randint(0, 3)
        if n_protocolos > 0:
            lst.append([idx, 'pct_s_' + str(pct), i[0], 
                        df.loc[idx, 'servico'], df.loc[idx, 'respondedor']])
            pct += 1
            for j in range(1, n_protocolos):
                dias = rd.randint(1, i[2])
                dt = i[0] + relativedelta(days = dias)
                lst.append([idx, 'pct_s_' + str(pct), dt, 
                            df.loc[idx, 'servico'], df.loc[idx, 'respondedor']])
                pct += 1
df_pct_se = pd.DataFrame(lst, columns=['idx', 'protocolo', 'dt_protocolo', 'servico', 'respondedor'])
lst_pct_se = list(df_pct_se['protocolo'])
display(df_pct_se)

,idx,protocolo,dt_protocolo,servico,respondedor
0,0,pct_s_1,2018-01-01,Questionário Semestral - SEDES,sedes
1,0,pct_s_2,2018-02-02,Questionário Semestral - SEDES,sedes
2,0,pct_s_3,2018-07-01,Questionário Semestral - SEDES,sedes
3,0,pct_s_4,2019-01-01,Questionário Semestral - SEDES,sedes
4,0,pct_s_5,2020-01-01,Questionário Semestral - SEDES,sedes
5,0,pct_s_6,2020-05-16,Questionário Semestral - SEDES,sedes
6,0,pct_s_7,2020-03-19,Questionário Semestral - SEDES,sedes
7,0,pct_s_8,2020-07-01,Questionário Semestral - SEDES,sedes
8,0,pct_s_9,2020-10-30,Questionário Semestral - SEDES,sedes
9,0,pct_s_10,2021-07-01,Questionário Semestral - SEDES,sedes


#### A.3.2 Campos Fictícios para servirços de respondedor SEDES - (A.3.1) x (4.1.8)

In [20]:
# gera campos fictícios a partir do dataset de protocolos e colunas fictícias
df_tmp = pd.merge(df_pct_se, df_campos_fic).drop(['idx', 'respondedor'], axis=1)

# atribui valor aos campos fictícios
df_tmp['valor'] = df_tmp['tipo'].apply(GeraValor)

# concatena os campos fictícios ao dataset principal
df_final = pd.concat([df_final, df_tmp])
df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_m_1,2018-07-01,Questionário Semestral - Município,col_1,STRING,valor_23
1,pct_m_1,2018-07-01,Questionário Semestral - Município,col_2,STRING,valor_102
2,pct_m_1,2018-07-01,Questionário Semestral - Município,col_3,STRING,valor_509
3,pct_m_1,2018-07-01,Questionário Semestral - Município,col_4,STRING,valor_328
4,pct_m_1,2018-07-01,Questionário Semestral - Município,col_5,STRING,valor_816
...,...,...,...,...,...,...
160,pct_s_11,2021-09-21,Questionário Semestral - SEDES,col_25,STRING,valor_699
161,pct_s_11,2021-09-21,Questionário Semestral - SEDES,col_26,STRING,valor_780
162,pct_s_11,2021-09-21,Questionário Semestral - SEDES,col_27,STRING,valor_607
163,pct_s_11,2021-09-21,Questionário Semestral - SEDES,col_28,STRING,valor_147


### A.3 NEGOCIO formulários Municípios

### E.1 Orgão Gestor

In [21]:
# TIPO 1 - 1 registro por protocolo
# métrica Órgão Gestor
colunas = ['protocolo', 'dt_protocolo', 'servico']
df_tmp = df_pct_mu[(df_pct_mu['servico'] == tup_servicos[0][0])][colunas]
df_tmp['atributo'] = ORGAO_GESTOR
df_tmp['tipo'] = 'STRING'
df_tmp['valor'] = df_tmp.apply(lambda row: rd.sample(lst_OG, k=1)[0], axis=1)

df_final = pd.concat([df_final, df_tmp])
df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_m_1,2018-07-01,Questionário Semestral - Município,col_1,STRING,valor_23
1,pct_m_1,2018-07-01,Questionário Semestral - Município,col_2,STRING,valor_102
2,pct_m_1,2018-07-01,Questionário Semestral - Município,col_3,STRING,valor_509
3,pct_m_1,2018-07-01,Questionário Semestral - Município,col_4,STRING,valor_328
4,pct_m_1,2018-07-01,Questionário Semestral - Município,col_5,STRING,valor_816
...,...,...,...,...,...,...
2659,pct_m_2660,2020-01-01,Questionário Semestral - Município,orgaoGestorDaPoliticaDeAssistenciaSocial,STRING,Orgão Gestor 02
2660,pct_m_2661,2020-05-21,Questionário Semestral - Município,orgaoGestorDaPoliticaDeAssistenciaSocial,STRING,Orgão Gestor 03
2661,pct_m_2662,2020-05-04,Questionário Semestral - Município,orgaoGestorDaPoliticaDeAssistenciaSocial,STRING,Orgão Gestor 04
2662,pct_m_2663,2021-07-01,Questionário Semestral - Município,orgaoGestorDaPoliticaDeAssistenciaSocial,STRING,Orgão Gestor 06


### E.2 Instrumentos de Gestão

In [22]:
# TIPO 2 - n registros por protocolo
# métrica Órgão Gestor
tup_vls = ('false', 'true')
lst_tmp = []
for i in lst_pct_mu:
    for k in TUP_INSTR_GESTAO:
        atributo = k
        tipo = 'STRING'
        valor = rd.sample(tup_vls, k=1)[0]
        lst_tmp.append([i, atributo, tipo, valor])

df_tmp = pd.DataFrame(lst_tmp, columns=['protocolo', 'atributo', 'tipo', 'valor'])
df_tmp = pd.merge(df_pct_mu, df_tmp).drop(['idx', 'municipio'], axis=1)

df_final = pd.concat([df_final, df_tmp])
df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_m_1,2018-07-01,Questionário Semestral - Município,col_1,STRING,valor_23
1,pct_m_1,2018-07-01,Questionário Semestral - Município,col_2,STRING,valor_102
2,pct_m_1,2018-07-01,Questionário Semestral - Município,col_3,STRING,valor_509
3,pct_m_1,2018-07-01,Questionário Semestral - Município,col_4,STRING,valor_328
4,pct_m_1,2018-07-01,Questionário Semestral - Município,col_5,STRING,valor_816
...,...,...,...,...,...,...
15979,pct_m_2664,2021-07-28,Questionário Semestral - Município,planoMunicipalDeAssistenciaSocial-possui,STRING,true
15980,pct_m_2664,2021-07-28,Questionário Semestral - Município,PlanoMunicipaldeEducacaoPermanentedoSUAS-possui,STRING,true
15981,pct_m_2664,2021-07-28,Questionário Semestral - Município,diagnosticoSocioterritorial-possui,STRING,false
15982,pct_m_2664,2021-07-28,Questionário Semestral - Município,RelatoriodeGestaoDoExercicioAnterior-possui,STRING,true


### E.3 Áreas Essenciais

In [23]:
# TIPO 2 - n registros por protocolo
# métrica Áreas Essenciais
tup_vls = ('false', 'true')
lst_tmp = []
for i in lst_pct_mu:
    for k in AREAS_ESSENCIAIS:
        atributo = k
        tipo = 'STRING'
        valor = rd.sample(tup_vls, k=1)[0]
        lst_tmp.append([i, atributo, tipo, valor])

df_tmp = pd.DataFrame(lst_tmp, columns=['protocolo', 'atributo', 'tipo', 'valor'])
df_tmp = pd.merge(df_pct_mu, df_tmp).drop(['idx', 'municipio'], axis=1)

df_final = pd.concat([df_final, df_tmp])
df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_m_1,2018-07-01,Questionário Semestral - Município,col_1,STRING,valor_23
1,pct_m_1,2018-07-01,Questionário Semestral - Município,col_2,STRING,valor_102
2,pct_m_1,2018-07-01,Questionário Semestral - Município,col_3,STRING,valor_509
3,pct_m_1,2018-07-01,Questionário Semestral - Município,col_4,STRING,valor_328
4,pct_m_1,2018-07-01,Questionário Semestral - Município,col_5,STRING,valor_816
...,...,...,...,...,...,...
10651,pct_m_2663,2021-07-01,Questionário Semestral - Município,quaisAreasEssenciaisDoSUASEstaoImplantadosOfic...,STRING,true
10652,pct_m_2664,2021-07-28,Questionário Semestral - Município,quaisAreasEssenciaisDoSUASEstaoImplantadosOfic...,STRING,true
10653,pct_m_2664,2021-07-28,Questionário Semestral - Município,quaisAreasEssenciaisDoSUASEstaoImplantadosOfic...,STRING,true
10654,pct_m_2664,2021-07-28,Questionário Semestral - Município,quaisAreasEssenciaisDoSUASEstaoImplantadosOfic...,STRING,false


### E.4 Organização PPA

In [24]:
# TIPO 1 - 1 registro por protocolo
# métrica Organização de PPA no município
colunas = ['protocolo', 'dt_protocolo', 'servico']
df_tmp = df_pct_mu[(df_pct_mu['servico'] == tup_servicos[0][0])][colunas]
df_tmp['atributo'] = ORGANIZACAO_PPA
df_tmp['tipo'] = 'STRING'
df_tmp['valor'] = df_tmp.apply(lambda row: rd.sample(lst_org_PPA, k=1)[0], axis=1)

df_final = pd.concat([df_final, df_tmp])
df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_m_1,2018-07-01,Questionário Semestral - Município,col_1,STRING,valor_23
1,pct_m_1,2018-07-01,Questionário Semestral - Município,col_2,STRING,valor_102
2,pct_m_1,2018-07-01,Questionário Semestral - Município,col_3,STRING,valor_509
3,pct_m_1,2018-07-01,Questionário Semestral - Município,col_4,STRING,valor_328
4,pct_m_1,2018-07-01,Questionário Semestral - Município,col_5,STRING,valor_816
...,...,...,...,...,...,...
2659,pct_m_2660,2020-01-01,Questionário Semestral - Município,comoEstaOrganizadoOPPADaPASNoMunicipio,STRING,Por nível de proteção social
2660,pct_m_2661,2020-05-21,Questionário Semestral - Município,comoEstaOrganizadoOPPADaPASNoMunicipio,STRING,Por nível de proteção social
2661,pct_m_2662,2020-05-04,Questionário Semestral - Município,comoEstaOrganizadoOPPADaPASNoMunicipio,STRING,"Por programas, serviços, projetos e benefícios"
2662,pct_m_2663,2021-07-01,Questionário Semestral - Município,comoEstaOrganizadoOPPADaPASNoMunicipio,STRING,"Por programas, serviços, projetos e benefícios"


### E.5 Vigilância Socioassistencial - VS

In [25]:
# TIPO 2 - n registros por protocolo
# métrica Vigilância SocioAssistencial
tup_vls = ('false', 'true')
lst_tmp = []
for i in lst_pct_mu:
    for k in VS:
        atributo = k
        tipo = 'STRING'
        valor = rd.sample(tup_vls, k=1)[0]
        lst_tmp.append([i, atributo, tipo, valor])

df_tmp = pd.DataFrame(lst_tmp, columns=['protocolo', 'atributo', 'tipo', 'valor'])
df_tmp = pd.merge(df_pct_mu, df_tmp).drop(['idx', 'municipio'], axis=1)

df_final = pd.concat([df_final, df_tmp])
df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_m_1,2018-07-01,Questionário Semestral - Município,col_1,STRING,valor_23
1,pct_m_1,2018-07-01,Questionário Semestral - Município,col_2,STRING,valor_102
2,pct_m_1,2018-07-01,Questionário Semestral - Município,col_3,STRING,valor_509
3,pct_m_1,2018-07-01,Questionário Semestral - Município,col_4,STRING,valor_328
4,pct_m_1,2018-07-01,Questionário Semestral - Município,col_5,STRING,valor_816
...,...,...,...,...,...,...
18643,pct_m_2664,2021-07-28,Questionário Semestral - Município,oMunicipioDesenvolveQuaisAcoesDeVigilanciaSoci...,STRING,true
18644,pct_m_2664,2021-07-28,Questionário Semestral - Município,oMunicipioDesenvolveQuaisAcoesDeVigilanciaSoci...,STRING,false
18645,pct_m_2664,2021-07-28,Questionário Semestral - Município,oMunicipioDesenvolveQuaisAcoesDeVigilanciaSoci...,STRING,false
18646,pct_m_2664,2021-07-28,Questionário Semestral - Município,oMunicipioDesenvolveQuaisAcoesDeVigilanciaSoci...,STRING,false


### E.6 Monitoramento da Alimentação de Dados da Rede SUAS

In [26]:
# TIPO 2 - n registros por protocolo
# Monitoramento de Sistemas
tup_vls = ('false', 'true')
lst_tmp = []
for i in lst_pct_mu:
    for k in MONITORA_SISTEMAS:
        atributo = PREFIXO_MONITORA_SISTEMAS + k
        tipo = 'STRING'
        valor = rd.sample(tup_vls, k=1)[0]
        lst_tmp.append([i, atributo, tipo, valor])

df_tmp = pd.DataFrame(lst_tmp, columns=['protocolo', 'atributo', 'tipo', 'valor'])
df_tmp = pd.merge(df_pct_mu, df_tmp).drop(['idx', 'municipio'], axis=1)

df_final = pd.concat([df_final, df_tmp])
df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_m_1,2018-07-01,Questionário Semestral - Município,col_1,STRING,valor_23
1,pct_m_1,2018-07-01,Questionário Semestral - Município,col_2,STRING,valor_102
2,pct_m_1,2018-07-01,Questionário Semestral - Município,col_3,STRING,valor_509
3,pct_m_1,2018-07-01,Questionário Semestral - Município,col_4,STRING,valor_328
4,pct_m_1,2018-07-01,Questionário Semestral - Município,col_5,STRING,valor_816
...,...,...,...,...,...,...
21307,pct_m_2664,2021-07-28,Questionário Semestral - Município,realizaMonitoramentoDaAlimentacaoDosDadosNosSi...,STRING,false
21308,pct_m_2664,2021-07-28,Questionário Semestral - Município,realizaMonitoramentoDaAlimentacaoDosDadosNosSi...,STRING,false
21309,pct_m_2664,2021-07-28,Questionário Semestral - Município,realizaMonitoramentoDaAlimentacaoDosDadosNosSi...,STRING,true
21310,pct_m_2664,2021-07-28,Questionário Semestral - Município,realizaMonitoramentoDaAlimentacaoDosDadosNosSi...,STRING,true


### E.7 Alimentação de Dados da Rede SUAS

In [27]:
# TIPO 2 - n registros por protocolo
# Alimentação de Dados no sistema da Rede SUAS
tup_vls = ('false', 'true')
lst_tmp = []
for i in lst_pct_mu:
    for k in ALIMENTA_SUAS:
        atributo = k
        tipo = 'STRING'
        valor = rd.sample(tup_vls, k=1)[0]
        lst_tmp.append([i, atributo, tipo, valor])

df_tmp = pd.DataFrame(lst_tmp, columns=['protocolo', 'atributo', 'tipo', 'valor'])
df_tmp = pd.merge(df_pct_mu, df_tmp).drop(['idx', 'municipio'], axis=1)

df_final = pd.concat([df_final, df_tmp])
df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_m_1,2018-07-01,Questionário Semestral - Município,col_1,STRING,valor_23
1,pct_m_1,2018-07-01,Questionário Semestral - Município,col_2,STRING,valor_102
2,pct_m_1,2018-07-01,Questionário Semestral - Município,col_3,STRING,valor_509
3,pct_m_1,2018-07-01,Questionário Semestral - Município,col_4,STRING,valor_328
4,pct_m_1,2018-07-01,Questionário Semestral - Município,col_5,STRING,valor_816
...,...,...,...,...,...,...
10651,pct_m_2663,2021-07-01,Questionário Semestral - Município,quemAlimentaOsDadosNosSistemasDaRedeSUAS-prote...,STRING,false
10652,pct_m_2664,2021-07-28,Questionário Semestral - Município,quemAlimentaOsDadosNosSistemasDaRedeSUAS-gestao,STRING,true
10653,pct_m_2664,2021-07-28,Questionário Semestral - Município,quemAlimentaOsDadosNosSistemasDaRedeSUAS-prote...,STRING,true
10654,pct_m_2664,2021-07-28,Questionário Semestral - Município,quemAlimentaOsDadosNosSistemasDaRedeSUAS-prote...,STRING,true


### E.8 Equipe que compõem setor de VS

In [28]:
def SorteiaEquipe():
    n = rd.randint(N_EQUIPE_VS[0], N_EQUIPE_VS[1])
    lst_tudo = []
    # monta a estrutura de profissionais e seus atributos
    for i in range(n):
        lst_linha = []
        lst_linha.append(names.get_full_name())
        lst_linha.append(rd.sample(lst_escolaridade, k=1)[0][0])
        lst_linha.append(rd.sample(lst_formacao, k=1)[0][0])
        lst_linha.append(rd.sample(lst_cargo, k=1)[0][0])
        lst_linha.append(rd.randint(1, 8))

        lst_tudo.append(lst_linha)
    
    return (lst_tudo)

In [29]:
# TIPO 3 - n registros e n colunas por protocolo
# estipula a quantidade de profissionais que vão compor a equipe do setor de VS
# para cada protocolo e para cada atributo estipula as colunas
lst_tmp = []
for i in lst_pct_mu:
    lst_sort = SorteiaEquipe()
    for idx, j in enumerate(lst_sort):
        
        for idx_k, k in enumerate(EQUIPE_VS):
            atributo = k['prefixo'] + '-' + str(idx) + '-' + k['sufixo']
            tipo = k['tipo']
            valor = j[idx_k]
            lst_tmp.append([i, atributo, tipo, valor])


df_tmp = pd.DataFrame(lst_tmp, columns=['protocolo', 'atributo', 'tipo', 'valor'])
df_tmp = pd.merge(df_pct_mu, df_tmp).drop(['idx', 'municipio'], axis=1)

df_final = pd.concat([df_final, df_tmp])
df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_m_1,2018-07-01,Questionário Semestral - Município,col_1,STRING,valor_23
1,pct_m_1,2018-07-01,Questionário Semestral - Município,col_2,STRING,valor_102
2,pct_m_1,2018-07-01,Questionário Semestral - Município,col_3,STRING,valor_509
3,pct_m_1,2018-07-01,Questionário Semestral - Município,col_4,STRING,valor_328
4,pct_m_1,2018-07-01,Questionário Semestral - Município,col_5,STRING,valor_816
...,...,...,...,...,...,...
53430,pct_m_2664,2021-07-28,Questionário Semestral - Município,qualEquipeCompoeOSetorDaVigilanciaSocioassiste...,STRING,Ahmad Whitaker
53431,pct_m_2664,2021-07-28,Questionário Semestral - Município,qualEquipeCompoeOSetorDaVigilanciaSocioassiste...,STRING,Médio - Completo
53432,pct_m_2664,2021-07-28,Questionário Semestral - Município,qualEquipeCompoeOSetorDaVigilanciaSocioassiste...,STRING,Psicologia
53433,pct_m_2664,2021-07-28,Questionário Semestral - Município,qualEquipeCompoeOSetorDaVigilanciaSocioassiste...,STRING,Especialistas


### E.9 Capacitações Ofertadas

In [30]:
# para cada protocolo e para cada atributo estipula as colunas
lst_co = []
for i in lst_pct_mu:
    for k in CAPACITACOES_OFERTADAS:
        atributo = k
        tipo = 'INTEGER'
        valor = rd.randint(N_CAPACITACOES_OFERTADAS[0], N_CAPACITACOES_OFERTADAS[1])
        lst_co.append([i, atributo, tipo, valor])

df_tmp = pd.DataFrame(lst_co, columns=['protocolo', 'atributo', 'tipo', 'valor'])
df_tmp = pd.merge(df_pct_mu, df_tmp).drop(['idx', 'municipio'], axis=1)

df_final = pd.concat([df_final, df_tmp])
df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_m_1,2018-07-01,Questionário Semestral - Município,col_1,STRING,valor_23
1,pct_m_1,2018-07-01,Questionário Semestral - Município,col_2,STRING,valor_102
2,pct_m_1,2018-07-01,Questionário Semestral - Município,col_3,STRING,valor_509
3,pct_m_1,2018-07-01,Questionário Semestral - Município,col_4,STRING,valor_328
4,pct_m_1,2018-07-01,Questionário Semestral - Município,col_5,STRING,valor_816
...,...,...,...,...,...,...
127867,pct_m_2664,2021-07-28,Questionário Semestral - Município,capacitacoesOfertadasProtecaoSocialEspecial-tr...,INTEGER,57
127868,pct_m_2664,2021-07-28,Questionário Semestral - Município,capacitacoesOfertadasProtecaoSocialEspecial-tr...,INTEGER,90
127869,pct_m_2664,2021-07-28,Questionário Semestral - Município,capacitacoesOfertadasProtecaoSocialEspecial-tr...,INTEGER,58
127870,pct_m_2664,2021-07-28,Questionário Semestral - Município,capacitacoesOfertadasProtecaoSocialEspecial-tr...,INTEGER,93


### E.10 Acompanhamento e assessoramento técnico estadual sobre o PAS

In [31]:
# para cada protocolo e para cada atributo estipula as colunas
lst_aco = []
for i in lst_pct_se:
    for k in ACOMP_ASSESS_PAS:
        atributo = k
        tipo = 'INTEGER'
        valor = rd.randint(N_ACOMP_ASSESS_PAS[0], N_ACOMP_ASSESS_PAS[1])
        lst_aco.append([i, atributo, tipo, valor])

df_tmp = pd.DataFrame(lst_aco, columns=['protocolo', 'atributo', 'tipo', 'valor'])
df_tmp = pd.merge(df_pct_se, df_tmp).drop(['idx', 'respondedor'], axis=1)

df_final = pd.concat([df_final, df_tmp])
df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_m_1,2018-07-01,Questionário Semestral - Município,col_1,STRING,valor_23
1,pct_m_1,2018-07-01,Questionário Semestral - Município,col_2,STRING,valor_102
2,pct_m_1,2018-07-01,Questionário Semestral - Município,col_3,STRING,valor_509
3,pct_m_1,2018-07-01,Questionário Semestral - Município,col_4,STRING,valor_328
4,pct_m_1,2018-07-01,Questionário Semestral - Município,col_5,STRING,valor_816
...,...,...,...,...,...,...
50,pct_s_11,2021-09-21,Questionário Semestral - SEDES,quantidadeDeMunicipiosAcompanhadosRemotamente,INTEGER,186
51,pct_s_11,2021-09-21,Questionário Semestral - SEDES,quantidadeDeMunicipiosVisitadosInLoco,INTEGER,108
52,pct_s_11,2021-09-21,Questionário Semestral - SEDES,quantidadeDeMunicipiosComPlanoDeProvidenciasAc...,INTEGER,122
53,pct_s_11,2021-09-21,Questionário Semestral - SEDES,quantidadeDeNotasTecnicasElaboradasEPublicizadas,INTEGER,187


### E.11 Atividades de Contrôle Social realizadas pelo Estado

In [32]:
# para cada protocolo e para cada atributo estipula as colunas
lst_aco = []
for i in lst_pct_se:
    for k in ACS_ESTADO:
        atributo = k
        tipo = 'INTEGER'
        valor = rd.randint(N_ACS_ESTADO[0], N_ACS_ESTADO[1])
        lst_aco.append([i, atributo, tipo, valor])

df_tmp = pd.DataFrame(lst_aco, columns=['protocolo', 'atributo', 'tipo', 'valor'])
df_tmp = pd.merge(df_pct_se, df_tmp).drop(['idx', 'respondedor'], axis=1)

df_final = pd.concat([df_final, df_tmp])
df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_m_1,2018-07-01,Questionário Semestral - Município,col_1,STRING,valor_23
1,pct_m_1,2018-07-01,Questionário Semestral - Município,col_2,STRING,valor_102
2,pct_m_1,2018-07-01,Questionário Semestral - Município,col_3,STRING,valor_509
3,pct_m_1,2018-07-01,Questionário Semestral - Município,col_4,STRING,valor_328
4,pct_m_1,2018-07-01,Questionário Semestral - Município,col_5,STRING,valor_816
...,...,...,...,...,...,...
39,pct_s_10,2021-07-01,Questionário Semestral - SEDES,quantidadeMensalDeReunioesTecnicasRealizadasPe...,INTEGER,3
40,pct_s_11,2021-09-21,Questionário Semestral - SEDES,quantidadeMensalDeReunioesExtraordinariasDoCEAS,INTEGER,8
41,pct_s_11,2021-09-21,Questionário Semestral - SEDES,quantidadeMensalDeReunioesOrdinariasDoCEAS,INTEGER,3
42,pct_s_11,2021-09-21,Questionário Semestral - SEDES,quantidadeMensalDeResolucoesEmitidasDoCEAS,INTEGER,9


# <font color='blue'>5.0 carga</font>

In [33]:
# grava tabelas
df_final.to_csv(caminho_out + 'dataset_sedes.csv', index=False)

In [34]:
# tipo 0-normal, 1-sqlalchemy
def ConectaBD(bd, tipo):
    
    s_host = 'localhost'
    s_bd   = bd
    s_user = 'gd' 
    s_pw   = 'Cavaquinho@Dourado@6390'
    
    if tipo == 0:
        cn = mysql.connector.connect(
            host        = s_host, 
            database    = s_bd, 
            user        = s_user, 
            password    = s_pw,
            auth_plugin = 'mysql_native_password'
        )
    elif tipo == 1:
        st = sa.engine.url.URL.create(
            drivername ='mysql+pymysql',
            username    =s_user,
            password    =s_pw,
            host        =s_host,
#             port='3307',
            database    =s_bd,
        )
        eng = sa.create_engine(st)
        cn = eng.connect()
    else:
        cn = None
        
    return cn

In [38]:
# grava dados de simulação no MySQL
# create sqlalchemy engine
    
print('LoadBD - carregando tabelas no BD ...')

try:
    dbConn = ConectaBD('bd_sedes_simula_fontes', 1)

    start_time = time.time()
    print('carregando tabela dataset_sedes... ', end='')
    df_final.to_sql('dataset_sedes', con=dbConn, if_exists='replace', index=False, chunksize = 10000)
    print('OK {0: .2f}'.format((time.time() - start_time)))

except ValueError as vx:
    print('ERROR -', vx)

except Exception as ex: 
    print('EXCEPTION -', ex)

else:
    print('Tabelas criadas com sucesso');  

finally:
    dbConn.close()

LoadBD - carregando tabelas no BD ...
carregando tabela dataset_sedes... OK  15.19
Tabelas criadas com sucesso


# TESTES